In [43]:
import pandas as pd 
import json 
import numpy as np 

import os
from tqdm import tqdm

### Grab the buildings identified within each image

In [44]:
dir_path = "train/labels/"
training_labels = os.listdir(dir_path)
sample = training_labels[0:100]

In [45]:
df = pd.DataFrame(columns = ['image_name', 'classification', 'uid', 'polygon'])
for path in sample: 
    d = pd.read_json(dir_path + path)
    # look to see if a feature is identified in this image 
    ll = d['features']['xy']
    if len(ll) > 0: 
        # look to see if any of these features are buildings 
        for i in range(0, len(ll)): 
            if ll[i]['properties']['feature_type'] == 'building' and 'subtype' in ll[i]['properties'].keys(): 
                add = {'image_name': training_labels[1], 'classification': ll[i]['properties']['subtype'], 'uid': ll[i]['properties']['uid'], 'polygon': ll[i]['wkt']}
                df.loc[len(df.index)] = add

In [46]:
df['classification'].value_counts()

no-damage        2473
minor-damage      358
destroyed         270
major-damage      180
un-classified      91
Name: classification, dtype: int64

In [47]:
df.head()

,image_name,classification,uid,polygon
0,midwest-flooding_00000308_post_disaster.json,no-damage,3350c4e6-e7dc-4313-8b23-59233a6e2078,"POLYGON ((319.9690811897951 118.8051167728774,..."
1,midwest-flooding_00000308_post_disaster.json,no-damage,4ac81fae-312a-4050-bd41-92f677311b09,"POLYGON ((448.3688195265795 92.6786134661785, ..."
2,midwest-flooding_00000308_post_disaster.json,no-damage,bcfa6989-a2b4-4aa6-a1f9-6adc62301b6c,"POLYGON ((862.4316509232316 13.3651563899852, ..."
3,midwest-flooding_00000308_post_disaster.json,no-damage,497560fe-6da5-4826-a443-cd76fa4f2f86,"POLYGON ((898.5940151494626 38.91158143295991,..."
4,midwest-flooding_00000308_post_disaster.json,no-damage,2d171f2b-c103-42d2-b1a4-8e954bbceaef,"POLYGON ((927.1242390329007 30.31084096671428,..."


### Restrict image to bounding box area 